In [25]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [26]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
masses = [2,4,12,15,16,18,28,32,40,44]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,10:].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.iloc[:, idx].dropna().values
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

               mass 2.0_time            mass 4.0_time  \
0    2023/05/10 16:40:31.854                      NaN   
1    2023/05/10 16:40:33.927                      NaN   
2    2023/05/10 16:40:39.001  2023/05/10 16:40:39.201   
3    2023/05/10 16:40:44.576  2023/05/10 16:40:44.776   
4    2023/05/10 16:40:50.151  2023/05/10 16:40:50.351   
..                       ...                      ...   
671  2023/05/10 17:42:48.676  2023/05/10 17:42:48.876   
672  2023/05/10 17:42:54.251  2023/05/10 17:42:54.451   
673  2023/05/10 17:42:59.826  2023/05/10 17:43:00.026   
674  2023/05/10 17:43:05.401  2023/05/10 17:43:05.601   
675  2023/05/10 17:43:10.976  2023/05/10 17:43:11.176   

              mass 12.0_time           mass 15.0_time  \
0                        NaN                      NaN   
1                        NaN                      NaN   
2    2023/05/10 16:40:39.401  2023/05/10 16:40:39.601   
3    2023/05/10 16:40:44.976  2023/05/10 16:40:45.176   
4    2023/05/10 16:40:50.551  

Index(['Timestamp, mass 2', 'Timestamp, mass 4', 'Timestamp, mass 12',
       'Timestamp, mass 15', 'Timestamp, mass 16', 'Timestamp, mass 18',
       'Timestamp, mass 28', 'Timestamp, mass 32', 'Timestamp, mass 40',
       'Timestamp, mass 44'],
      dtype='object')

#### Vaclogger

In [27]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-10 15:23:33,0.0,-0.021565,1.920000e-09,7.750000e-10,4.996000e-09,5.996000e-07,-0.000788,0.0,46.352,113.580,0.000007,0.000000,NaN,NaN
1,NaN,2023-05-10 15:23:42,9.0,-0.021246,1.930000e-09,7.770000e-10,4.996000e-09,5.903000e-07,-0.000777,0.0,46.352,113.692,0.000006,0.000006,NaN,NaN
2,NaN,2023-05-10 15:23:50,17.0,-0.021246,1.920000e-09,7.800000e-10,4.996000e-09,5.882000e-07,-0.000763,0.0,46.240,113.580,0.000006,0.000006,NaN,NaN
3,NaN,2023-05-10 15:23:59,26.0,-0.021467,1.930000e-09,7.840000e-10,4.996000e-09,5.826000e-07,-0.000772,0.0,46.352,113.467,0.000007,0.000006,NaN,NaN
4,NaN,2023-05-10 15:24:08,35.0,-0.021392,1.920000e-09,7.890000e-10,4.996000e-09,5.763000e-07,-0.000755,0.0,46.352,113.580,0.000006,0.000006,NaN,NaN


In [28]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-05-10 16:43:23,10/05/2023,0.000,16:43:23.524,0.0,0.06,0.08,0.1,0.0,0.0,...,0.0098,0.0096,0.0135,0.0121,0.0026,-0.0144,NaN,NaN,9.500000e-09,1.280000e-08
1,2023-05-10 16:43:24,10/05/2023,1.180,16:43:24.704,0.0,0.06,0.08,0.1,0.0,0.0,...,0.0098,0.0096,0.0135,0.0121,0.0026,-0.0144,NaN,NaN,9.500000e-09,1.280000e-08
2,2023-05-10 16:43:25,10/05/2023,2.362,16:43:25.886,0.0,0.06,0.08,0.1,0.0,0.0,...,0.0098,0.0096,0.0135,0.0121,0.0026,-0.0144,NaN,NaN,9.500000e-09,1.280000e-08
3,2023-05-10 16:43:27,10/05/2023,3.544,16:43:27.068,0.0,0.06,0.08,0.1,0.0,0.0,...,0.0098,0.0096,0.0135,0.0121,0.0026,-0.0144,NaN,NaN,9.500000e-09,1.280000e-08
4,2023-05-10 16:43:28,10/05/2023,4.715,16:43:28.239,0.0,0.06,0.08,0.1,0.0,0.0,...,0.0098,0.0096,0.0135,0.0121,0.0026,-0.0144,NaN,NaN,9.500000e-09,1.280000e-08


## Data processing

#### Comments

In [29]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'starting LHe transfer'
 'Transfer complete & waiting for stable temperature'
 'ESD measurement start' 'end']


In [30]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'no emission' 'grid repelling' 'grid transparent' 'grid repelling 2'
 'grid transparent 2' 'grid repelling 3' 'grid transparent 3'
 'grid repelling 4' 'grid transparent 4' 'grid repelling 5' 'emission off']


Int64Index([   5,    6,    7,    8,   45,   46,   47,   48,   49,  118,  119,
             120,  504,  505,  506,  507, 1032, 1033, 1034, 1035, 1140, 1141,
            1142, 1417, 1418, 1419, 1420, 1421, 1596, 1597, 1598, 1599, 2369,
            2370, 2371, 2372, 2373, 2619, 2620, 2621, 2622, 2733, 2734, 2735],
           dtype='int64')

#### CernOx temperature R-T conversion

In [31]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [32]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [33]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(12,6))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S2 ESD @30-40K CU EGA halfway - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
""" 
#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)

 """
plt.savefig(r'./graphs/MID.png')
plt.show()


In [34]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S2 ESD @30-40K EGA halway - HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta.png')
plt.show()


In [35]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
#plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S2 ESD @30-40K CU, EGA halfway - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Pressures.png')
plt.show()

In [36]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S2 ESD @30-40K CU EGA halfway - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()